In [7]:
import numpy as np
import pandas as pd
import sqlite3 as sql
from sklearn.preprocessing import MinMaxScaler
from ipywidgets import interact ## para análisis interactivo
from sklearn import neighbors ### basado en contenido un solo producto consumido
import joblib
#### conectar_base_de_Datos

conn=sql.connect('data\\db_movies2.db')
cur=conn.cursor()

#### ver tablas disponibles en base de datos ###

cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
cur.fetchall()

[('ratings',),
 ('movies',),
 ('usuarios_sel',),
 ('movies_sel',),
 ('ratings_final',),
 ('full_ratings',),
 ('full_ratings_sin_dup',),
 ('reco',)]

In [ ]:
######################################################################
################## 1. sistemas basados en popularidad ###############
#####################################################################

In [10]:
##### recomendaciones basado en popularidad ######
## peliculas vistas por más de 20 personas y con calificación mayor a 4
pd.read_sql("""
            SELECT movie_title,
                   avg(rating) as avg_rat,
                   count(*) as movie_num
            FROM full_ratings
            WHERE rating > 4  -- Restringir a calificaciones mayores a 4
            GROUP BY movie_title
            HAVING count(*) >= 20
            ORDER BY avg_rat DESC, movie_num DESC
            LIMIT 10
            """, conn)

,movie_title,avg_rat,movie_num
0,Jaws (1975),4.968750,32
1,Babe (1995),4.964286,28
2,Sense and Sensibility (1995),4.950000,20
3,Speed (1994),4.940000,25
4,Apocalypse Now (1979),4.931818,44
5,Lawrence of Arabia (1962),4.931818,22
6,Beauty and the Beast (1991),4.926471,34
7,Brazil (1985),4.925000,20
8,"Bridge on the River Kwai, The (1957)",4.925000,20
9,Leaving Las Vegas (1995),4.925000,20


In [9]:
#### los mejores calificados por año publicacion ###
pd.read_sql("""SELECT Año, movie_title, num_vistas
                FROM (
                    SELECT
                        strftime('%Y', fecha_nueva) AS Año,
                        movie_title,
                        ROW_NUMBER() OVER(PARTITION BY strftime('%Y', fecha_nueva) ORDER BY COUNT(*) DESC) AS row_num,
                        MAX(COUNT(*)) OVER(PARTITION BY strftime('%Y', fecha_nueva)) AS num_vistas
                    FROM full_ratings
                    GROUP BY strftime('%Y', fecha_nueva), movie_title) AS subquery
                WHERE row_num = 1
                ORDER BY Año;""", conn)

,Año,movie_title,num_vistas
0,1996,Batman (1989),83
1,1997,Independence Day (a.k.a. ID4) (1996),27
2,1998,Air Force One (1997),6
3,1999,Saving Private Ryan (1998),15
4,2000,American Beauty (1999),27
5,2001,Star Wars: Episode IV - A New Hope (1977),13
6,2002,"Silence of the Lambs, The (1991)",11
7,2003,"Lord of the Rings: The Two Towers, The (2002)",14
8,2004,Austin Powers: The Spy Who Shagged Me (1999),9
9,2005,"Sixth Sense, The (1999)",14
